# Desafio Parte 3

## Tarefa 3: Processamento da Trusted

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame

## Define o bucket S3 de origem e o caminho do arquivo para os dados JSON
source_bucket = "data-lake-da-telma"
source_file_path = "Raw/TMDB/JSON/Superhero/2023/04/13/superhero.json"
collection_date = "2023/04/13"


## Crea GlueContext e SparkContext
sc = SparkContext()
glueContext = GlueContext(sc)

## Crea a DynamicFrame dos dados de origem no formato JSON
source_dyf = glueContext.create_dynamic_frame_from_options(
    "s3",
    {"paths": [f"s3://{source_bucket}/{source_file_path}"]},
    "json"
)

## Define o bucket S3 de destino e o caminho do arquivo para os dados do Parquet na zona Truested
destination_bucket = "data-lake-da-telma"
destination_file_path = f"Trusted/{collection_date}.parquet"

## Converte DynamicFrame em um DataFrame
source_df = source_dyf.toDF()

## Converte o DataFrame em um DynamicFrame no formato Parquet
destination_dyf = DynamicFrame.fromDF(
    source_df,
    glueContext,
    "parquet"
)

## Salva o DynamicFrame no formato Parquet no bucket S3 de destino com caminho do arquivo na zona trusted
glueContext.write_dynamic_frame.from_options(
    frame = destination_dyf,
    connection_type = "s3",
    connection_options = {"path": f"s3://{destination_bucket}/{destination_file_path}"},
    format = "parquet"
)
